## Portfolio and Risk Management

This project is used as a tool to practice and apply methods I have learned about Portfolio and Risk Management. This project will be expanded on a rolling basis, mostly limited by how much time I have after school, etc... 
First we need to get some data.

In [1]:
## Imports

import matplotlib as plt
import pandas as pd
import numpy as np
import time
from datetime import datetime
import yfinance as yf

In [2]:
## Tickers part of the portfolio
tickers = ['TSLA', 'GME', 'AMC' ,'MSFT', 'GOOG']

portfolio = pd.read_csv('Data/Portfolio.csv', index_col = 0)

portfolio.columns = tickers

The next step would be to check for completness of the data and familiarize yourself with the structure of the data.

In [3]:
portfolio.head()

,TSLA,GME,AMC,MSFT,GOOG
Date,,,,,
2016-01-04,44.681999,21.848803,17.572334,49.591064,741.840027
2016-01-05,44.686001,22.203815,17.777107,49.817295,742.580017
2016-01-06,43.807999,21.895105,17.595087,48.912342,743.619995
2016-01-07,43.130001,21.956850,17.155214,47.211044,726.390015
2016-01-08,42.200001,21.895105,16.973190,47.355843,714.469971


In [4]:
portfolio.describe()

,TSLA,GME,AMC,MSFT,GOOG
count,1521.000000,1521.000000,1521.000000,1521.000000,1521.000000
mean,221.454803,40.280898,16.201018,135.963729,1336.691101
std,291.042571,63.279808,10.567634,80.806940,612.532663
min,28.733999,2.800000,1.980000,44.449951,668.260010
25%,50.310001,10.100000,9.489752,67.510132,930.390015
50%,65.820000,15.945121,13.260000,107.105705,1140.989990
75%,286.152008,22.180662,22.130543,200.470337,1485.930054
max,1229.910034,347.510010,62.549999,343.109985,3014.179932


In [5]:
portfolio.isnull().values.any()

False

It also seems to be helpful to plot the plot prices as a function of time. However, since some of the assets have largely different prices the scale might be affected.

In [6]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

pio.renderers.default = "svg"

fig = px.line(portfolio)

fig.update_layout(legend_title = "Asset", title = "Asset Prices over Time")
fig.update_xaxes(title = "Date")
fig.update_yaxes(title = "Stock Prices (in USD)")
fig.update_layout(width = 950, height = 700, title_x = 0.5, template = "ggplot2")

fig.write_image("Images/Asset_Prices_Over_Time.png")

![title](Images/Asset_Prices_Over_Time.png)

As expected, the scale suffers. Let's plot the cummulative returns. This should increase legibility of the plot.


In [7]:
asset_returns = portfolio.pct_change().dropna()

asset_returns.head()

,TSLA,GME,AMC,MSFT,GOOG
Date,,,,,
2016-01-05,0.000090,0.016249,0.011653,0.004562,0.000998
2016-01-06,-0.019648,-0.013903,-0.010239,-0.018165,0.001400
2016-01-07,-0.015477,0.002820,-0.025000,-0.034783,-0.023170
2016-01-08,-0.021563,-0.002812,-0.010610,0.003067,-0.016410
2016-01-11,-0.014929,0.035601,-0.004021,-0.000573,0.002184


In [8]:
asset_returns_cum = ((1+asset_returns).cumprod()-1) * 100

asset_returns_cum.head()

,TSLA,GME,AMC,MSFT,GOOG
Date,,,,,
2016-01-05,0.008956,1.624862,1.165314,0.456192,0.099751
2016-01-06,-1.956046,0.211924,0.129481,-1.368638,0.239940
2016-01-07,-3.473430,0.494524,-2.373731,-4.799292,-2.082661
2016-01-08,-5.554806,0.211924,-3.409587,-4.507308,-3.689482
2016-01-11,-6.964772,3.779568,-3.798007,-4.562054,-3.479186


In [9]:
fig = px.line(asset_returns_cum)

fig.update_layout(legend_title = "Asset", title = "Cummulative Return over Time")
fig.update_xaxes(title = "Date")
fig.update_yaxes(title = "Cummulative Return (in %)")
fig.update_layout(width = 950, height = 700, title_x = 0.5, template = "ggplot2")

fig.write_image("Images/Cum_Return_Over_Time.png")

![title](Images/Cum_Return_Over_Time.png)

This plot helps a little more. From the first plot we would expect GOOG to be performing the best. However, this plot clearly shows that TSLA is blowing GOOG out of the water. As is GME, which seemed to be underperforming significantly when compared to GOOG. Let's take a closer look at the historical data of TSLA.

## TSLA Risk Analysis

In [10]:
tsla_asset = portfolio['TSLA']
tsla_return = tsla_asset.pct_change().dropna() * 100

In [11]:
fig = px.line(tsla_return)
fig.update_layout(legend_title = "Asset", title = "Returns (in %) per Day of TSLA")
fig.update_xaxes(title = "Date")
fig.update_yaxes(title = "Return (in %)")
fig.update_layout(width = 950, height = 700, title_x = 0.5, template = "ggplot2")

fig.write_image("Images/Returns_Per_Day_TSLA.png")

![title](Images/Returns_Per_Day_TSLA.png)

From the return per day we can see that the beginning of 2020 was very eventful. This can be explained by the COVID-19 pandemic. Then, the recent visible increase in volatility can also explained by the fear caused by the COVID-19 omicron variant. Let's calculate volatility.

In [12]:
import math

tsla_daily_volatility = tsla_return.std()
print("The average daily volatility is", tsla_daily_volatility)

tsla_monthly_volatility = math.sqrt(21) * tsla_daily_volatility
print("The average monthly volatility is", tsla_monthly_volatility)

tsla_annual_volatility = math.sqrt(252) * tsla_daily_volatility
print("The average annual volatility is", tsla_annual_volatility)

The average daily volatility is 3.625287137572878
The average monthly volatility is 16.613152723877498
The average annual volatility is 57.54964918331424


From here it seems useful to take a look at the distribution of the returns as well as the VaR and cVaR.

In [13]:
fig = px.histogram(tsla_return)
fig.update_layout(legend_title = "", title = "Returns (in %) of TSLA")
fig.update_xaxes(title = "Returns (in %)")
fig.update_yaxes(title = "Frequency of Returns")
fig.update_layout(width = 950, height = 700, title_x = 0.5, template = "ggplot2")

fig.write_image("Images/Returns_TSLA_Hist.png")

![title](Images/Returns_TSLA_Hist.png)

As can be seen, the returns are relatively normal distributed (possibly t-distribution). For now, we are going to assume a normal distribution:

In [14]:
## Calculate VaR(90) and CVaR(90)
var_90 = np.percentile(tsla_return, 10)
cvar_90 = tsla_return[tsla_return <= var_90].mean()

## Calculate VaR(95) and CVaR(95)
var_95 = np.percentile(tsla_return, 5)
cvar_95 = tsla_return[tsla_return <= var_95].mean()

## Calculate VaR(99) and CVaR(99)
var_99 = np.percentile(tsla_return, 1)
cvar_99 = tsla_return[tsla_return <= var_99].mean()


print("VaR(90):",var_90)
print("CVaR(90):", cvar_90)

print("")

print("VaR(90):",var_95)
print("CVaR(90):", cvar_95)

print("")

print("VaR(99):",var_99)
print("CVaR(99):", cvar_99)

VaR(90): -3.354413502899729
CVaR(90): -5.983841024719736

VaR(90): -4.977261277312514
CVaR(90): -7.902930921500541

VaR(99): -8.974423869669101
CVaR(99): -13.277236397085636


Let's add these values to the histogram!

In [15]:
fig.update_yaxes(range = [0,150])

fig.add_trace(go.Scatter(x=(var_90,var_90), y = (0,200), mode = 'lines', line_color='Red', name = "VaR(90)"))
fig.add_trace(go.Scatter(x=(cvar_90,cvar_90),y = (0,200), line_dash='dash', line_color='Red',name = "CVaR(90)"))


fig.add_trace(go.Scatter(x=(var_95,var_95), y = (0,200), mode = 'lines', line_color='Green', name = "VaR(95)"))
fig.add_trace(go.Scatter(x=(cvar_95,cvar_95),y = (0,200), line_dash='dash', line_color='Green',name = "CVaR(95)"))

fig.add_trace(go.Scatter(x=(var_99,var_99), y = (0,200), mode = 'lines', line_color='Purple', name = "VaR(99)"))
fig.add_trace(go.Scatter(x=(cvar_99,cvar_99),y = (0,200), line_dash='dash', line_color='Purple',name = "CVaR(99)"))

fig.write_image("Images/Returns_TSLA_Hist_VaR.png")

![title](Images/Returns_TSLA_Hist_VaR.png)

Great, this plot is giving us a lot of information. Let's focus only on VaR(90). The value is telling us that (historically) there is a probability of 90 percent that the losses don't exceed -3.35%. If they do, there is a 90% probabilty the losses don't exceed 5.98%. This is an example of very simple and straight forward risk management. But before we dive into more detail only focusing on the TSLA assest, let's look at our whole portfolio.

## Hedging

Hedging is mostly used to offset any losses. At first, we will look at how the asset price has an influence on the cost of a put option. To do so we will use the Black Scholes method of pricing options.

In [16]:
## Defining Black Scholes functions 

# The Black-Scholes option pricing formula for European options
# Adapted from: Black, F., & Scholes, M. (1973). The Pricing of Options and Corporate Liabilities. Journal of Political Economy, 81(3), 637-654, www.jstor.org/stable/1831029.
# MIT License
# Copyright (c) 2019 Jamsheed Shorish

import numpy as np
from scipy.stats import norm


# Auxiliary function for d_one risk-adjusted probability
def d11(S, X, T, r, sigma):
    """This is an auxiliary function and is not intended to be called externally."""
    return (np.log(S/X) + (r + 0.5 * sigma**2)*T) / (sigma * np.sqrt(T))

# Auxiliary function for d_two risk-adjusted probability    
def d21(d1, T, sigma):
    """This is an auxiliary function and is not intended to be called externally."""
    return d1 - sigma * np.sqrt(T)

# Definition of the Black-Scholes delta function    
def bs_delta(S, X, T, r, sigma, option_type):
    """Compute the delta of the Black-Scholes option pricing formula.
    
    Arguments:
    S           -- the current spot price of the underlying stock
    X           -- the option strike price
    T           -- the time until maturity (in fractions of a year)
    r           -- the risk-free interest rate 
    sigma       -- the returns volatility of the underlying stock
    option_type -- the option type, either 'call' or 'put'
    
    Returns: a numpy.float_ representing the delta value
    Exceptions raised: ValueError if option_type is not 'call' or 'put'
    """
    if option_type == 'call':
        return norm.cdf(d11(S, X, T, r, sigma))
    elif option_type == 'put':
        return norm.cdf(-d11(S, X, T, r, sigma))
    else:
        # Raise an error if the option_type is neither a call nor a put
        raise ValueError("Option type is either 'call' or 'put'.")

# Definition of the Black-Scholes European option pricing formula    
def black_scholes(S, X, T, r, sigma, option_type):
    """Price a European option using the Black-Scholes option pricing formula.
    
    Arguments:
    S           -- the current spot price of the underlying stock
    X           -- the option strike price
    T           -- the time until maturity (in fractions of a year)
    r           -- the risk-free interest rate 
    sigma       -- the returns volatility of the underlying stock
    option_type -- the option type, either 'call' or 'put'
    
    Returns: a numpy.float_ representing the option value
    Exceptions raised: ValueError if option_type is not 'call' or 'put'
    """
    d_one = d11(S, X, T, r, sigma)
    d_two = d21(d_one, T, sigma)
    if option_type == 'call':
        return S * norm.cdf(d_one) - np.exp(-r * T) * X * norm.cdf(d_two)
    elif option_type == 'put':
        return -(S * norm.cdf(-d_one) - np.exp(-r * T) * X * norm.cdf(-d_two))
    else:
        # Raise an error if the option_type is neither a call nor a put
        raise ValueError("Option type is either 'call' or 'put'.")

## Defining Black Scholes functions

In [17]:
tsla_price_last_year = tsla_asset.tail(252)
option_values_1500 = np.zeros(tsla_price_last_year.size)

## Strike price is choosen arbitrarily

for i,S in enumerate(tsla_price_last_year.values):
    option_values_1500[i] = black_scholes(S = S, X = 1500, T = 1, r = 0.02, 
                        sigma = tsla_annual_volatility/100, option_type = 'put')

In [18]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.array(range(252)), y = option_values_1500, mode = 'lines',
                         name = "Put Option:<br>Strike = $1500"))

fig.add_trace(go.Scatter(x=np.array(range(252)), y = tsla_price_last_year, mode = 'lines', name = "TSLA"))
fig.update_layout(legend_title = "Asset", title = "Correlation between Asset Price and Put Value")
fig.update_xaxes(title = "Time")
fig.update_yaxes(title = "Value (in $)")
fig.update_layout(width = 950, height = 700, title_x = 0.5, template = "ggplot2")

fig.write_image("Images/TSLA_Put_New.png")

![title](Images/TSLA_Put_New.png)

Here we can clearly see the connection between the assets price and the put price. If the asset prices rises, the value of the put option lowers and vice versa. This is exactly what we need for our hedging plan.

Now we can calculate our option delta. This will allow us to show that with each change in the asset/option price we offset our loss. Let's assume our assets value falls from 1100 to 1095.

In [19]:
value = black_scholes(S = 1100, X = 1500, T = 1, r = 0.02, 
                      sigma = tsla_annual_volatility/100, option_type = "put")

delta = bs_delta(S = 1100, X = 1500, T = 1, r = 0.02, 
                 sigma = tsla_annual_volatility/100, option_type = "put")

value_change = black_scholes(S = 1095, X = 1500, T = 1, r = 0.02, 
                             sigma = tsla_annual_volatility/100, option_type = "put") - value

print( (1095 - 1100) + (1/delta) * value_change )

0.013151183222512408


Generally the aim is for delta neutrality = 0 - this would mean we offset any drop in the assets price with the same increase in the put value. We can see that this is the case in our example for TSLA. Therefore, this put option would be very useful to hedge our TSLA stock.

## Best Suited Portfolio Weights - GMV and MSR and The Efficient Frontier

Let's determine the portfolio weights for the best valued portfolio and the portfolio with the least risk - we can use the efficient frontier to do so! We are going to include all assets in the efficient frontier.

In [20]:
annualized_asset_returns = (1+asset_returns.mean())**252 - 1
annualized_cov = (asset_returns).cov()*252
risk_free_rate=0
n_portfolios = 2000

plot_pairs = []
weights_list = []

np.random.seed(2)

for i in range(n_portfolios):
    weights = np.random.rand(5)
    weights = weights/sum(weights)
    weights_list.append([weights])
    portfolio_E_Variance = np.sqrt(np.dot(weights.T, np.dot(annualized_cov, weights)))
    portfolio_E_Return = annualized_asset_returns.multiply(weights, axis = 0).sum()
    plot_pairs.append([portfolio_E_Return, portfolio_E_Variance])
    
    
plot_pairs = np.array(plot_pairs)
sharpe_ratio = np.array((plot_pairs[:,0] - risk_free_rate)/plot_pairs[:,1])
fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_pairs[:,1], y=plot_pairs[:,0], 
                         marker = dict(color = sharpe_ratio,showscale=True,
                                       colorbar=dict(title="Sharpe<br>Ratio")),mode='markers'))
fig.update_layout(title = "The Efficient Frontier",title_x = 0.5)
fig.update_yaxes(title = "Annualized Return")
fig.update_xaxes(title = "Annualized Risk (Volatility)")
fig.write_image("Images/The_Efficient_Frontier_New.png")

![title](Images/The_Efficient_Frontier_New.png)

From here we can see the efficient frontier. We can pick a point on the efficient frontier following the basic rule: The higher the return, the higher the risk. The lower the return, the lower the risk. Let's calculate the MSR and GMV.

In [21]:
indx_msr = sharpe_ratio.argsort()[len(sharpe_ratio)-1]

print("The maximum sharpe ratio is:", sharpe_ratio[indx_msr])

print("The associated weights are:",weights_list[indx_msr])

The maximum sharpe ratio is: 2.1940235623317452
The associated weights are: [array([0.39209185, 0.14340418, 0.00617222, 0.445952  , 0.01237976])]


In [22]:
indx_gmv = plot_pairs[:,1].argsort()[0]

print("The minimum volatility is:",plot_pairs[indx_gmv][1])

print("The associated weights are:",weights_list[indx_gmv])

The minimum volatility is: 0.2502319371101916
The associated weights are: [array([0.02928756, 0.04248366, 0.01379509, 0.4640012 , 0.45043249])]


In [23]:
print(tickers)

['TSLA', 'GME', 'AMC', 'MSFT', 'GOOG']


This is giving us some great insight. If we want a great performing portfolio (high sharpe ratio) we would wanna hold mostly MSFT and TSLA with some GME but almost no GOOG and GME. However, this sharpe ratio is historical. This means, just because the historical sharpe ratio is good - the future is not guaranteed to be good. When looking at the GMV, we would mostly hold MSTF and GOOG. This makes sense since the historical growth has been really steady for both stocks - indicating low risk - but also low reward. 

Let's asses our VaR and CVaR again! 

In [24]:
gmv_weights = [0.02928756, 0.04248366, 0.01379509, 0.4640012 , 0.45043249]

gmv_portfolio = (portfolio * gmv_weights).sum(axis=1)
gmv_portfolio_pct_returns = gmv_portfolio.pct_change().dropna() * 100


In [25]:
fig = px.line(gmv_portfolio_pct_returns)
fig.update_layout(legend_title = "Asset", title = "Returns (in %) per Day of our GMV")
fig.update_xaxes(title = "Date")
fig.update_yaxes(title = "Return (in %)")
fig.update_layout(width = 950, height = 700, title_x = 0.5, template = "ggplot2",showlegend=False)

fig.write_image("Images/Returns_Per_Day_GMV_New.png")

![title](Images/Returns_Per_Day_GMV_New.png)

We have choosen the GMV this means that the volatility overall is very low. However, there is a high volatility peak around the beginning of 2020 which is unavoidable due to COVID-19.

In [26]:
fig = px.histogram(gmv_portfolio_pct_returns)
fig.update_layout(legend_title = "", title = "Returns (in %) of GMV")
fig.update_xaxes(title = "Returns (in %)")
fig.update_yaxes(title = "Frequency of Returns")
fig.update_layout(width = 950, height = 700, title_x = 0.5, template = "ggplot2", showlegend = False)

fig.write_image("Images/Returns_GMV_Hist_New.png")

![title](Images/Returns_GMV_Hist_New.png)

Let's add our *historical* VaR and CVaR.

In [27]:
## Calculate VaR(90) and CVaR(90)
var_90 = np.percentile(gmv_portfolio_pct_returns, 10)
cvar_90 = tsla_return[gmv_portfolio_pct_returns <= var_90].mean()

## Calculate VaR(95) and CVaR(95)
var_95 = np.percentile(gmv_portfolio_pct_returns, 5)
cvar_95 = tsla_return[gmv_portfolio_pct_returns <= var_95].mean()

## Calculate VaR(99) and CVaR(99)
var_99 = np.percentile(gmv_portfolio_pct_returns, 1)
cvar_99 = tsla_return[gmv_portfolio_pct_returns <= var_99].mean()


print("VaR(90):",var_90)
print("CVaR(90):", cvar_90)

print("")

print("VaR(90):",var_95)
print("CVaR(90):", cvar_95)

print("")

print("VaR(99):",var_99)
print("CVaR(99):", cvar_99)

fig.update_yaxes(range = [0,150])

fig.add_trace(go.Scatter(x=(var_90,var_90), y = (0,200), mode = 'lines', line_color='Red', name = "VaR(90)"))
fig.add_trace(go.Scatter(x=(cvar_90,cvar_90),y = (0,200), line_dash='dash', line_color='Red',name = "CVaR(90)"))


fig.add_trace(go.Scatter(x=(var_95,var_95), y = (0,200), mode = 'lines', line_color='Green', name = "VaR(95)"))
fig.add_trace(go.Scatter(x=(cvar_95,cvar_95),y = (0,200), line_dash='dash', line_color='Green',name = "CVaR(95)"))

fig.add_trace(go.Scatter(x=(var_99,var_99), y = (0,200), mode = 'lines', line_color='Purple', name = "VaR(99)"))
fig.add_trace(go.Scatter(x=(cvar_99,cvar_99),y = (0,200), line_dash='dash', line_color='Purple',name = "CVaR(99)"))

fig.write_image("Images/Returns_GMV_Hist_VaR.png")

VaR(90): -1.6208245377309805
CVaR(90): -2.9433353757639957

VaR(90): -2.4769765312070344
CVaR(90): -3.726762755595857

VaR(99): -4.7602899224502755
CVaR(99): -5.731114971323832


![title](Images/Returns_GMV_Hist_VaR.png)

However, this does not help us a lot. We know historically, it is expected to not loose more than 1.62% in 95% of cases. But this is the past - we need the future. We can use a Monte Carlo Estimation to generate possible futures and use those to calculate a better VaR and CVaR. First we need the daily portfolio variance.

In [28]:
portfolio_returns = portfolio.pct_change()
portfolio_cov = portfolio_returns.cov()
portfolio_vol_daily = np.sqrt(np.dot(np.dot(np.array(gmv_weights), portfolio_cov), np.array(gmv_weights).T))
portfolio_avg_ret = np.mean(gmv_portfolio_pct_returns)

In [29]:
print("The average daily return is:",portfolio_avg_ret, "%")
print("The average daily volatility is:",portfolio_vol_daily*100,"%")

The average daily return is: 0.10411631037108311 %
The average daily volatility is: 1.5763130415588282 %


If we compare that to the daily volatility of the TSLA stock we see a large difference (1.57% (GMV) vs. 3.62% (TSLA)). If assuming that the distribution of the returns in the future is normal, we can choose mu to be our average daily portfolio return and the standard deviation to be the daily volatility. We are going to predict 252 days (one trading year). Then we can simulate the returns:

In [30]:
mu = portfolio_avg_ret
sd = portfolio_vol_daily*100
T = 252

sim_returns = []

## Each simulation is one year
for i in range(100):

    rand_rets = np.random.normal(mu, sd, T)
    
    sim_returns = np.concatenate([sim_returns, rand_rets])

We can use the simulated returns to calculate the most probable VaR(95) and CVaR(95).

In [31]:
## Calculate VaR(90) and CVaR(90)
var_95 = np.percentile(sim_returns, 5)
cvar_95 = sim_returns[sim_returns <= var_95].mean()

print("VaR(95):",var_95)
print("CVaR(95):", cvar_95)

VaR(95): -2.475763717610942
CVaR(95): -3.1388585560309212


We can see that using the Monte Carlo estimation we get similar VaR(95) and CVaR(95) values for the *upcoming* year -  which makes sense.

## The GARCH Model for Prediction and VaR/CVaR

TO BE IMPLEMENTED

## Predicting Asset Prices using Neural Networks and Deep Learning

In this section we will use 4 of our stock prices to predict the price of the last/5th one using Deep Learning and Tensorflow. 

For our training set we will keep all prices except the last - most recent - 6 Month. This month can be used for testing. Let's predict the price of GOOG.

In [33]:
train_set = portfolio.drop(portfolio.tail(21*6).index)
test_set = portfolio.tail(21*6)

X_train = train_set[['TSLA','GME','AMC','MSFT']]
y_train = train_set[['GOOG']]

X_test = test_set[['TSLA','GME','AMC','MSFT']]
y_test = test_set[['GOOG']]

After creating the train and test set we can prepare the model and train it.

In [39]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()
model.add(layers.Dense(500, input_dim=4, activation='sigmoid'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(25, activation='relu'))
model.add(layers.Dense(1))

model.compile(loss='mean_squared_logarithmic_error', optimizer='rmsprop')
model.fit(X_train, y_train, epochs=10000)

2022-01-24 09:17:35.573506: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


44/44 [==============================] - 1s 9ms/step - loss: 7.1102


In [35]:
pred = pd.DataFrame(model.predict(X_test)).set_index(y_test.index)
pred.columns = ["GOOG (Pred)"]
pred_df = pd.concat([y_test, pred], axis = 1)

2022-01-24 09:16:48.749897: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [37]:
plot_df = pd.concat([X_test,pred_df], axis = 1)

fig = px.line(plot_df)

fig.update_layout(legend_title = "Asset", title = "Asset Prices over Time ('GOOG' Predicted with Deep Learning)")
fig.update_xaxes(title = "Date")
fig.update_yaxes(title = "Stock Prices (in USD)")
fig.update_layout(width = 950, height = 700, title_x = 0.5, template = "ggplot2")

fig.write_image('Images/Asset_Prices_Predicted_Deep_1000.png')

![title](Images/Asset_Prices_Predicted_Deep_1000.png)

Here we can see that our model does not perform *tooooo* well. It seems to underestimate the price. That is better than overestimating since that could potentially lead to heavy losses. However, the model could be improved by adding layers and epochs! For now, this is more or less only a proof of concept.